# 8.5 Exercises
# 8.1 OneorTwoGroups
- Paired t-Test and Wilcoxon signed rank sum test
> The daily energy intake from 11 healthy women is [5260., 5470., 5640., 6180., 6390., 6515., 6805., 7515., 7515., 8230., 8770.] kJ.
> Is this value significantly different from the recommended value of 7725? (Correct answer: yes, pttest D 0:018, and pWilcoxon D 0:026.)
- t-Test of independent samples
> In a clinic, 15 lazy patients weigh [76, 101, 66, 72, 88, 82, 79, 73, 76, 85, 75, 64, 76, 81, 86.] kg, and 15 sporty patients weigh [ 64, 65, 56, 62, 59, 76, 66, 82, 91, 57, 92, 80, 82, 67, 54] kg.
> Are the lazy patients significantly heavier? (Correct answer: yes, p D 0:045.)
- Normality test
> Are the two data sets normally distributed? (Correct answer: yes, they are.)
- Mann–Whitney test
> Are the lazy patients still heavier, if you check with the Mann–Whitney test? (Correct answer: no, p D 0:077. Note, however, that the answer would be "yes" for a one-sided test!)

In [2]:
'''Solution for Exercise "Comparing Groups" '''
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
import os

def oneGroup():
    '''Test of mean value of a single set of data'''
    
    print('Single group of data =========================================')
    
    # First get the data
    data = np.array([5260, 5470, 5640, 6180, 6390, 6515, 6805, 7515, 7515, 8230, 8770], dtype=np.float)
    checkValue = 7725   # value to compare the data to
    
    # 4.1.1. Normality test
    # We don't need the first parameter, so we just assign the output to the dummy variable "_"
    (_, p) = stats.normaltest(data)
    if p > 0.05:
        print('Data are distributed normally, p = {0}'.format(p))
        
    # 4.1.2. Do the onesample t-test
    t, prob = stats.ttest_1samp(data, checkValue)
    if prob < 0.05:
        print('With the one-sample t-test, {0:4.2f} is significantly different from the mean (p={1:5.3f}).'.\
        format(checkValue, prob))
    else:
        print('No difference from reference value with onesample t-test.')
    
    # 4.1.3. This implementation of the Wilcoxon test checks for the "difference" of one vector of data from zero
    (_,p) = stats.wilcoxon(data-checkValue)
    if p < 0.05:
        print('With the Wilcoxon test, {0:4.2f} is significantly different from the mean (p={1:5.3f}).'.\
        format(checkValue, p))
    else:
        print('No difference from reference value with Wilcoxon rank sum test.')
    
def twoGroups():
    '''Compare the mean of two groups'''
    
    print('Two groups of data =========================================')

    # Enter the data
    data1 = [76., 101., 66., 72., 88., 82., 79., 73., 76., 85., 75., 64., 76., 81., 86.]
    data2 = [64., 65., 56., 62., 59., 76., 66., 82., 91., 57., 92., 80., 82., 67., 54.]
    
    # Normality test 
    print('\n Normality test --------------------------------------------------')
    
    # To do the test for both data-sets, make a tuple with "(... , ...)", add a counter with
    # "enumerate", and and iterate over the set:
    for ii, data in enumerate((data1, data2)):
        (_, pval) = stats.normaltest(data)
        if pval > 0.05:
            print('Dataset # {0} is normally distributed'.format(ii))
    
    # T-test of independent samples 
    print('\n T-test of independent samples -------------------------------------------')
    
    # Do the t-test for independent samples
    t, pval = stats.ttest_ind(data1, data2)
    if pval < 0.05:
        print('With the T-test, data1 and data2 are significantly different (p = {0:5.3f})'.format(pval))
    else:
        print('No difference between data1 and data2 with T-test.')
    
    # Mann-Whitney test --------------------------------------------------------------
    print('\n Mann-Whitney test --------------------------------------------------------')
    # Watch out: the keyword "alternative" was introduced in scipy 0.17, with default"two-sided";
    if np.int(sp.__version__.split('.')[1]) > 16:
        u, pval = stats.mannwhitneyu(data1, data2, alternative='two-sided')
    else:
        u, pval = stats.mannwhitneyu(data1, data2)
        pval *= 2    # because the default was a one-sided p-value

    if pval < 0.05:
        print('With the Mann-Whitney test, data1 and data2 are significantly different(p = {0:5.3f})'.format(pval))
    else:
        print('No difference between data1 and data2 with Mann-Whitney test.')

if __name__ == '__main__':
    oneGroup()    
    twoGroups()
    
    # And at the end, make sure the results are displayed, even if the program is run from the commandline
    input('\nDone! Thanks for using programs by thomas.\nHit any key to finish.')

Single group of data =========================================
Data are distributed normally, p = 0.6813132824061632
With the one-sample t-test, 7725.00 is significantly different from the mean (p=0.018).
With the Wilcoxon test, 7725.00 is significantly different from the mean (p=0.026).
Two groups of data =========================================

 Normality test --------------------------------------------------
Dataset # 0 is normally distributed
Dataset # 1 is normally distributed

 T-test of independent samples -------------------------------------------
With the T-test, data1 and data2 are significantly different (p = 0.045)

 Mann-Whitney test --------------------------------------------------------
No difference between data1 and data2 with Mann-Whitney test.

Done! Thanks for using programs by thomas.
Hit any key to finish.


# 8.2 Multiple Groups
- The following example is taken from the really good, but somewhat advanced book by A.J. Dobson: “An Introduction to Generalized Linear Models”:
- Get the data
> The file Data/data_others/Table 6.6 Plant experiment.xls, which can also be found on https://github.com/thomas-haslwanter/statsintro/tree/master/Data/ data_others, contains data from an experiment with plants in three different growing conditions. Read the data into Python. Hint: use the module xlrd.
- Perform an ANOVA
> Are the three groups different? (Correct answer: yes, they are.)
- Multiple Comparisons
> Using the Tukey test, which of the pairs are different? (Correct answer: only TreamtmentA and TreatmentB differ.)
- Kruskal–Wallis
> Would a nonparametric comparison lead to a different result? (Correct answer: no.

In [14]:
''' Solution for Exercise "Comparing Multiple Groups" '''

# author: Thomas Haslwanter, date: Sept-2015

# Load the required modules -----------------------------------------------------------
# Standard modules
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
%matplotlib inline
# Modules for data-analysis
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats import multicomp

# Module for working with Excel-files
import xlrd

def get_ANOVA_data():
    '''Get the data for the ANOVA'''

    # First we have to get the Excel-data into Python. This can be done e.g. with the package "xlrd"
    # You have to make sure that you select a valid location on your computer!
    inFile = 'Table 6.6 Plant experiment.xls'
    book = xlrd.open_workbook(inFile)
    # We assume that the data are in the first sheet. This avoids the language problem "Tabelle/Sheet"
    sheet = book.sheet_by_index(0)
    
    # Select the columns and rows that you want:
    # The "treatment" information is in column "E", i.e. you have to skip the first 4 columns
    # The "weight" information is in column "F", i.e. you have to skip the first 5 columns
    treatment = sheet.col_values(4)
    weight = sheet.col_values(5)
    
    # The data start in line 4, i.e. you have to skip the first 3
    # I use a "pandas" DataFrame, so that I can assign names to the variables.
    data = pd.DataFrame({'group':treatment[3:], 'weight':weight[3:]})
    
    return data

def do_ANOVA(data):
    '''4.3.2. Perform an ANOVA on the data'''
    
    print('ANOVA: ----------------------------------------------')
    
    # First, I fit a statistical "ordinary least square (ols)"-model to the data, using the
    # formula language from "patsy". The formula 'weight ~ C(group)' says:
    # "weight" is a function of the categorical value "group"
    # and the data are taken from the DataFrame "data", which contains "weight" and "group"
    model = ols('weight ~ C(group)', data).fit()
    
    # "anova_lm" (where "lm" stands for "linear model") extracts the ANOVA-parameters
    # from the fitted model.
    anovaResults = anova_lm(model)
    print(anovaResults)
    
    if anovaResults['PR(>F)'][0] < 0.05:
        print('One of the groups is different.')
        
def compare_many(data):
    '''Multiple comparisons: Which one is different? '''
    
    print('\n MultComp: --------------------------------------')
    
    # An ANOVA is a hypothesis test, and only answers the question: "Are all the groups 
    # from the same distribution?" It does not tell you which one is different.
    # Since we now compare many different groups to each other, we have to adjust the
    # p-values to make sure that we don't get a Type I error. For this, we use the 
    # statscom module "multicomp"
    mc = multicomp.MultiComparison(data['weight'], data['group'])
    
    # There are many ways to do multiple comparisons. Here, we choose
    # "Tukeys Honest Significant Difference" test
    # The first element of the output ("0") is a table containing the results
    print(mc.tukeyhsd().summary())
    
    # Show the group names
    print(mc.groupsunique)
    
    # Generate a print ----------------
    
    res2 = mc.tukeyhsd()     # Get the data
    
    simple = False
    if simple:
        # You can do the plot with a one-liner, but then this does not - yet - look that great
        res2.plot_simultaneous()
    else:
        # Or you can do it the hard way, i.e. by hand:
        
        # Plot values and errorbars
        xvals = np.arange(3)
        plt.plot(xvals, res2.meandiffs, 'o')
        errors = np.ravel(np.diff(res2.confint)/2)
        plt.errorbar(xvals, res2.meandiffs, yerr=errors, fmt='o')
        
        # Set the x-limits
        xlim = -0.5, 2.5
        # The "*xlim" passes the elements of the variable "xlim" elementwise into the function "hlines"
        plt.hlines(0, *xlim)
        plt.xlim(*xlim)
        
        # Plot labels (this is a bit tricky):
        # First, "np.array(mc.groupsunique)" makes an array with the names of the groups;
        # and then, "np.column_stack(res2[1][0])]" puts the correct groups together
        pair_labels = mc.groupsunique[np.column_stack(res2._multicomp.pairindices)]
        plt.xticks(xvals, pair_labels)
        
        plt.title('Multiple Comparison of Means - Tukey HSD, FWER=0.05' +
        '\n Pairwise Mean Differences')
    
    plt.show()

def KruskalWallis(data):
    '''Non-parametric comparison between the groups'''
    
    print('\n Kruskal-Wallis test ----------------------------------------------------')
    
    # First, I get the values from the dataframe
    g_a = data['weight'][data['group']=='TreatmentA']
    g_b = data['weight'][data['group']=='TreatmentB']
    g_c = data['weight'][data['group']=='Control']
    
    #Note: this could also be accomplished with the "groupby" function from pandas
    #groups = pd.groupby(data, 'group')
    #g_a = groups.get_group('TreatmentA').values[:,1]
    #g_c = groups.get_group('Control').values[:,1]
    #g_b = groups.get_group('TreatmentB').values[:,1]
    
    # Then do the Kruskal-Wallis test
    h, p = stats.kruskal(g_c, g_a, g_b)
    print('Result from Kruskal-Wallis test: p = {0}'.format(p))

if __name__ == '__main__':
    data = get_ANOVA_data()
    do_ANOVA(data)
    KruskalWallis(data)
    compare_many(data)
    input('\nThanks for using programs by Thomas!\nHit any key to finish')

ANOVA: ----------------------------------------------
            df    sum_sq   mean_sq         F   PR(>F)
C(group)   2.0   3.76634  1.883170  4.846088  0.01591
Residual  27.0  10.49209  0.388596       NaN      NaN
One of the groups is different.

 Kruskal-Wallis test ----------------------------------------------------
Result from Kruskal-Wallis test: p = 0.018423755731471966

 MultComp: --------------------------------------
 Multiple Comparison of Means - Tukey HSD,FWER=0.05 
  group1     group2   meandiff  lower  upper  reject
----------------------------------------------------
 Control   TreatmentA  -0.371  -1.0621 0.3201 False 
 Control   TreatmentB  0.494   -0.1971 1.1851 False 
TreatmentA TreatmentB  0.865    0.1739 1.5561  True 
----------------------------------------------------
['Control' 'TreatmentA' 'TreatmentB']


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1117fc598> (for post_execute):


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

<Figure size 432x288 with 1 Axes>